# Training Multiple ML Models:

The whole purpose of this notebook is to train mutliple ML models on the datasets that we have and then choosing the best dataset and the best model we got!

This will be done by dataset where each dataset is fitted into multiple models and at the end we get the best parameters per model and the best scores and from the scores we decude on which dataset and which model to use.

## Full One Hot Encoding:

In [1]:
# opening the csv and fixing the dt:

import pandas as pd
df = pd.read_csv("../data/ml-datasets/onehot_3.csv")
df['Last Updated'] = pd.to_datetime(df['Last Updated'])

In [2]:
# general look at our dataset:

df.head()

,Unnamed: 0,App,Rating,Reviews,Size,Label,Paid,Price,Last Updated,Action,...,TOOLS,TRAVEL AND LOCAL,VIDEO PLAYERS,WEATHER,Adults only 18+,Everyone,Everyone 10+,Mature 17+,Teen,Unrated
0,8532,+Download 4 Instagram Twitter,4.5,40467,22000,1,0,0.0,2018-08-02,False,...,False,False,False,False,False,True,False,False,False,False
1,324,- Free Comics - Comic Apps,3.5,115,91000,0,0,0.0,2018-07-13,False,...,False,False,False,False,False,False,False,True,False,False
2,4541,.R,4.5,259,203,0,0,0.0,2014-09-16,False,...,True,False,False,False,False,True,False,False,False,False
3,4636,/u/app,4.7,573,53000,0,0,0.0,2018-07-03,False,...,False,False,False,False,False,False,False,True,False,False
4,5940,058.ba,4.4,27,14000,0,0,0.0,2018-07-06,False,...,False,False,False,False,False,True,False,False,False,False


In [3]:
# dropping the column that we won't need in the dataset for the training:

df = df.drop(columns=["App", "Reviews", "Rating", "Unnamed: 0", ])

In [4]:
# making sure that the columns were dropped and cheking dtypes:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 98 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Size                     8190 non-null   int64         
 1   Label                    8190 non-null   int64         
 2   Paid                     8190 non-null   int64         
 3   Price                    8190 non-null   float64       
 4   Last Updated             8190 non-null   datetime64[ns]
 5   Action                   8190 non-null   bool          
 6   Adventure                8190 non-null   bool          
 7   Arcade                   8190 non-null   bool          
 8   Art & Design             8190 non-null   bool          
 9   Auto & Vehicles          8190 non-null   bool          
 10  Beauty                   8190 non-null   bool          
 11  Board                    8190 non-null   bool          
 12  Books & Reference        8190 non-

In [5]:
# turning the bool columns from True/False to 1/0:

bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

In [6]:
int32_cols = df.select_dtypes(include='int64').columns
int32_cols

Index(['Size', 'Label', 'Paid', 'Action', 'Adventure', 'Arcade',
       'Art & Design', 'Auto & Vehicles', 'Beauty', 'Board',
       'Books & Reference', 'Business', 'Card', 'Casino', 'Casual', 'Comics',
       'Communication', 'Dating', 'Education', 'Educational', 'Entertainment',
       'Events', 'Finance', 'Food & Drink', 'Health & Fitness', 'House & Home',
       'Libraries & Demo', 'Lifestyle', 'Maps & Navigation', 'Medical',
       'Music', 'Music & Audio', 'News & Magazines', 'Parenting',
       'Personalization', 'Photography', 'Productivity', 'Puzzle', 'Racing',
       'Role Playing', 'Shopping', 'Simulation', 'Social', 'Sports',
       'Strategy', 'Tools', 'Travel & Local', 'Trivia',
       'Video Players & Editors', 'Weather', 'Word', 'Action & Adventure',
       'Brain Games', 'Creativity', 'Education.1', 'Music & Video',
       'Pretend Play', 'ART AND DESIGN', 'AUTO AND VEHICLES', 'BEAUTY',
       'BOOKS AND REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION', 'DATING',
   

In [7]:
# opening the dataset we will be using to maintain the category distribution between the training set and the testing set

df_full = pd.read_csv("../data/ml-datasets/full_label.csv")
df_full

,Unnamed: 0,App,Category,Rating,Reviews,Size,Label,Paid,Price,Content Rating,Genres,Last Updated
0,8532,+Download 4 Instagram Twitter,SOCIAL,4.5,40467,22000,1,0,0.00,Everyone,Social,2018-08-02
1,324,- Free Comics - Comic Apps,COMICS,3.5,115,91000,0,0,0.00,Mature 17+,Comics,2018-07-13
2,4541,.R,TOOLS,4.5,259,203,0,0,0.00,Everyone,Tools,2014-09-16
3,4636,/u/app,COMMUNICATION,4.7,573,53000,0,0,0.00,Mature 17+,Communication,2018-07-03
4,5940,058.ba,NEWS AND MAGAZINES,4.4,27,14000,0,0,0.00,Everyone,News & Magazines,2018-07-06
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,6420,감성학원 BL 첫사랑,COMICS,4.4,190,34000,0,0,0.00,Everyone,Comics,2017-11-17
8186,4362,💎 I'm rich,LIFESTYLE,3.8,718,26000,0,1,399.99,Everyone,Lifestyle,2018-03-11
8187,2575,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.6,22098,18000,1,0,0.00,Everyone,Social,2018-07-24
8188,7559,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.0,19,32000,0,0,0.00,Everyone,Tools,2017-10-21


In [8]:
df_full['Last Updated'] = pd.to_datetime(df_full['Last Updated'])

In [9]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Unnamed: 0      8190 non-null   int64         
 1   App             8190 non-null   object        
 2   Category        8190 non-null   object        
 3   Rating          8190 non-null   float64       
 4   Reviews         8190 non-null   int64         
 5   Size            8190 non-null   int64         
 6   Label           8190 non-null   int64         
 7   Paid            8190 non-null   int64         
 8   Price           8190 non-null   float64       
 9   Content Rating  8190 non-null   object        
 10  Genres          8190 non-null   object        
 11  Last Updated    8190 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(5), object(4)
memory usage: 767.9+ KB


In [10]:
df_full = df_full.sort_values(by='Last Updated')
df_full = df_full.drop(columns="Unnamed: 0")

In [11]:
df_full

,App,Category,Rating,Reviews,Size,Label,Paid,Price,Content Rating,Genres,Last Updated
3390,FML F*ck my life + widget,FAMILY,4.2,1415,209,1,0,0.00,Everyone,Entertainment,2010-05-21
1645,CJ Poker Odds Calculator,GAME,4.1,207,116,0,0,0.00,Everyone,Card,2011-01-30
3332,FD Shift Calendar Widget,TOOLS,4.1,981,73,1,0,0.00,Everyone,Tools,2011-03-16
2323,DEER HUNTER CHALLENGE,GAME,3.7,38767,41000,1,0,0.00,Everyone 10+,Action,2011-04-11
3748,Freecell CY,GAME,4.0,387,11000,0,0,0.00,Everyone,Card,2011-04-16
...,...,...,...,...,...,...,...,...,...,...,...
1125,BankNordik,FINANCE,3.9,28,15000,0,0,0.00,Everyone,Finance,2018-08-08
7478,Video Downloader For FB: Save FB Videos 2018,TOOLS,4.3,434,36000,0,0,0.00,Everyone,Tools,2018-08-08
6514,Shoot Hunter-Gun Killer,GAME,4.3,320334,27000,1,0,0.00,Teen,Action,2018-08-08
4680,Lalafo Pulsuz Elanlar,SHOPPING,4.4,61392,0,1,0,0.00,Everyone,Shopping,2018-08-08


In [12]:
# importing StandardScalar to standardize the numeric values that we

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [13]:
# standardizing the numeric values:

df[["Size", "Price"]] = ss.fit_transform(df[["Size", "Price"]])

In [14]:
# making the training and testinf setts based on the dataframe we had that had the categories in one column:

from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_full, test_size=0.2, stratify=df_full['Category'], random_state=42)

In [15]:
train_indices = train_df.index
test_indices = test_df.index

In [16]:
# getting the train and test set in our main dataset based on the dataset that had the categories in one column

df = df.sort_values(by='Last Updated').drop(columns="Last Updated")
train_df = df.loc[train_indices]
test_df = df.loc[test_indices]

In [17]:
X_train = train_df.drop(columns=['Label'])
y_train = train_df["Label"]

X_test = test_df.drop(columns=['Label'])
y_test = test_df["Label"]

#### the models:

In [18]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
import numpy as np

# Define the classifiers
lr = LogisticRegression(random_state=42)
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
ada = AdaBoostClassifier()
xgb = XGBClassifier(eval_metric='logloss')

# Define parameter grids
para_knn = {'n_neighbors': np.arange(1, 50)}
para_dt = {'criterion': ['gini', 'entropy'], 'max_depth': np.arange(1, 50),
           'min_samples_leaf': [1, 2, 4, 5, 10, 20, 30, 40, 50, 80, 100]}
params_rf = {'n_estimators': [100, 200, 350, 500],
             'min_samples_leaf': [2, 10, 30, 50, 80, 100]}
params_ada = {'n_estimators': [50, 100, 250, 400, 500],
              'learning_rate': [0.1, 0.001, 0.2, 0.5, 0.8, 1],
              "algorithm":["SAMME"]}
params_xgb = {'n_estimators': [50, 100, 250, 600, 800, 1000],
              'learning_rate': [0.1, 0.001, 0.2, 0.5, 0.8, 1]}

# Function to perform CV and log all results
def cross_validation(model, param_grid, X_train, y_train, search_type='grid', model_name=""):
        if search_type == 'grid':
            search = GridSearchCV(model, param_grid=param_grid, cv=5, return_train_score=True)
        else:
            search = RandomizedSearchCV(model, param_distributions=param_grid, cv=5, return_train_score=True)
        search.fit(X_train, y_train)
        print(f"Best parameters for {model_name}: {search.best_params_}")
        print(f"Best cross-validated score for {model_name}: {search.best_score_}")

# Run CV searches and log all results
cross_validation(knn, para_knn, X_train, y_train, model_name="KNN")
cross_validation(dt, para_dt, X_train, y_train, model_name="Decision Tree")
cross_validation(rf, params_rf, X_train, y_train, model_name="Random Forest")
cross_validation(ada, params_ada, X_train, y_train, model_name="AdaBoost")
cross_validation(xgb, params_xgb, X_train, y_train, search_type='random', model_name="XGBoost")


ModuleNotFoundError: No module named 'xgboost'

# One Hot and Label Encoding:

In [ ]:
import pandas as pd
df = pd.read_csv("../data/ml-datasets/onehot_label_3.csv")
df['Last Updated'] = pd.to_datetime(df['Last Updated'])

In [ ]:
df

,Unnamed: 0,App,Category,Rating,Reviews,Size,Label,Paid,Price,Last Updated,...,Creativity,Education.1,Music & Video,Pretend Play,Adults only 18+,Everyone,Everyone 10+,Mature 17+,Teen,Unrated
0,8532,+Download 4 Instagram Twitter,SOCIAL,4.5,40467,22000,1,0,0.00,2018-08-02,...,False,False,False,False,False,True,False,False,False,False
1,324,- Free Comics - Comic Apps,COMICS,3.5,115,91000,0,0,0.00,2018-07-13,...,False,False,False,False,False,False,False,True,False,False
2,4541,.R,TOOLS,4.5,259,203,0,0,0.00,2014-09-16,...,False,False,False,False,False,True,False,False,False,False
3,4636,/u/app,COMMUNICATION,4.7,573,53000,0,0,0.00,2018-07-03,...,False,False,False,False,False,False,False,True,False,False
4,5940,058.ba,NEWS AND MAGAZINES,4.4,27,14000,0,0,0.00,2018-07-06,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8185,6420,감성학원 BL 첫사랑,COMICS,4.4,190,34000,0,0,0.00,2017-11-17,...,False,False,False,False,False,True,False,False,False,False
8186,4362,💎 I'm rich,LIFESTYLE,3.8,718,26000,0,1,399.99,2018-03-11,...,False,False,False,False,False,True,False,False,False,False
8187,2575,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.6,22098,18000,1,0,0.00,2018-07-24,...,False,False,False,False,False,True,False,False,False,False
8188,7559,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.0,19,32000,0,0,0.00,2017-10-21,...,False,False,False,False,False,True,False,False,False,False


In [ ]:
df = df.drop(columns=["App", "Reviews", "Rating", "Unnamed: 0", ])

In [ ]:
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df[["Size", "Price"]] = ss.fit_transform(df[["Size", "Price"]])

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder()
df["Category"] = label_enc.fit_transform(df["Category"])

In [ ]:
df.head()

,Category,Size,Label,Paid,Price,Last Updated,Action,Adventure,Arcade,Art & Design,...,Creativity,Education.1,Music & Video,Pretend Play,Adults only 18+,Everyone,Everyone 10+,Mature 17+,Teen,Unrated
0,27,-0.437196,1,0,-0.061575,2018-08-02,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,5,2.114768,0,0,-0.061575,2018-07-13,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,29,-1.243357,0,0,-0.061575,2014-09-16,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,6,0.709339,0,0,-0.061575,2018-07-03,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,21,-0.733075,0,0,-0.061575,2018-07-06,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
df_full = pd.read_csv("../data/ml-datasets/full_label.csv")
df_full

,Unnamed: 0,App,Category,Rating,Reviews,Size,Label,Paid,Price,Content Rating,Genres,Last Updated
0,8532,+Download 4 Instagram Twitter,SOCIAL,4.5,40467,22000,1,0,0.00,Everyone,Social,2018-08-02
1,324,- Free Comics - Comic Apps,COMICS,3.5,115,91000,0,0,0.00,Mature 17+,Comics,2018-07-13
2,4541,.R,TOOLS,4.5,259,203,0,0,0.00,Everyone,Tools,2014-09-16
3,4636,/u/app,COMMUNICATION,4.7,573,53000,0,0,0.00,Mature 17+,Communication,2018-07-03
4,5940,058.ba,NEWS AND MAGAZINES,4.4,27,14000,0,0,0.00,Everyone,News & Magazines,2018-07-06
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,6420,감성학원 BL 첫사랑,COMICS,4.4,190,34000,0,0,0.00,Everyone,Comics,2017-11-17
8186,4362,💎 I'm rich,LIFESTYLE,3.8,718,26000,0,1,399.99,Everyone,Lifestyle,2018-03-11
8187,2575,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.6,22098,18000,1,0,0.00,Everyone,Social,2018-07-24
8188,7559,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.0,19,32000,0,0,0.00,Everyone,Tools,2017-10-21


In [ ]:
df_full['Last Updated'] = pd.to_datetime(df_full['Last Updated'])
df_full = df_full.sort_values(by='Last Updated')
df_full = df_full.drop(columns="Unnamed: 0")

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_full, test_size=0.2, stratify=df_full['Category'], random_state=42)
train_indices = train_df.index
test_indices = test_df.index
df = df.sort_values(by='Last Updated').drop(columns="Last Updated")
train_df = df.loc[train_indices]
test_df = df.loc[test_indices]
X_train = train_df.drop(columns=['Label'])
y_train = train_df["Label"]

X_test = test_df.drop(columns=['Label'])
y_test = test_df["Label"]

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
import numpy as np

# Define the classifiers
lr = LogisticRegression(random_state=42)
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
ada = AdaBoostClassifier()
xgb = XGBClassifier(eval_metric='logloss')

# Define parameter grids
para_knn = {'n_neighbors': np.arange(1, 50)}
para_dt = {'criterion': ['gini', 'entropy'], 'max_depth': np.arange(1, 50),
           'min_samples_leaf': [1, 2, 4, 5, 10, 20, 30, 40, 50, 80, 100]}
params_rf = {'n_estimators': [100, 200, 350, 500],
             'min_samples_leaf': [2, 10, 30, 50, 80, 100]}
params_ada = {'n_estimators': [50, 100, 250, 400, 500],
              'learning_rate': [0.1, 0.001, 0.2, 0.5, 0.8, 1],
              "algorithm":["SAMME"]}
params_xgb = {'n_estimators': [50, 100, 250, 600, 800, 1000],
              'learning_rate': [0.1, 0.001, 0.2, 0.5, 0.8, 1]}

def cross_validation(model, param_grid, X_train, y_train, search_type='grid', model_name=""):
        if search_type == 'grid':
            search = GridSearchCV(model, param_grid=param_grid, cv=5, return_train_score=True)
        else:
            search = RandomizedSearchCV(model, param_distributions=param_grid, cv=5, return_train_score=True)
        search.fit(X_train, y_train)
        print(f"Best parameters for {model_name}: {search.best_params_}")
        print(f"Best cross-validated score for {model_name}: {search.best_score_}")

# Run CV searches and log all results
cross_validation(knn, para_knn, X_train, y_train, model_name="KNN")
cross_validation(dt, para_dt, X_train, y_train, model_name="Decision Tree")
cross_validation(rf, params_rf, X_train, y_train, model_name="Random Forest")
cross_validation(ada, params_ada, X_train, y_train, model_name="AdaBoost")
cross_validation(xgb, params_xgb, X_train, y_train, search_type='random', model_name="XGBoost")


Best parameters for KNN: {'n_neighbors': 17}
Best cross-validated score for KNN: 0.6913905241031553
Best parameters for Decision Tree: {'criterion': 'gini', 'max_depth': 18, 'min_samples_leaf': 10}
Best cross-validated score for Decision Tree: 0.6967348507345363
Best parameters for Random Forest: {'min_samples_leaf': 2, 'n_estimators': 100}
Best cross-validated score for Random Forest: 0.708484403840667
Best parameters for AdaBoost: {'algorithm': 'SAMME', 'learning_rate': 1, 'n_estimators': 500}
Best cross-validated score for AdaBoost: 0.6909321594726944
Best parameters for XGBoost: {'n_estimators': 600, 'learning_rate': 0.1}
Best cross-validated score for XGBoost: 0.706197005956644


# No Genres Column:

In [ ]:
df = pd.read_csv("../data/ml-datasets/no_genres_3.csv")
df['Last Updated'] = pd.to_datetime(df['Last Updated'])

In [ ]:
df = df.drop(columns=["App", "Reviews", "Rating", "Unnamed: 0", ])
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df[["Size", "Price"]] = ss.fit_transform(df[["Size", "Price"]])

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder()
df["Category"] = label_enc.fit_transform(df["Category"])

In [ ]:
df

,Category,Size,Label,Paid,Price,Last Updated,Adults only 18+,Everyone,Everyone 10+,Mature 17+,Teen,Unrated
0,27,-0.437196,1,0,-0.061575,2018-08-02,0,1,0,0,0,0
1,5,2.114768,0,0,-0.061575,2018-07-13,0,0,0,1,0,0
2,29,-1.243357,0,0,-0.061575,2014-09-16,0,1,0,0,0,0
3,6,0.709339,0,0,-0.061575,2018-07-03,0,0,0,1,0,0
4,21,-0.733075,0,0,-0.061575,2018-07-06,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,5,0.006624,0,0,-0.061575,2017-11-17,0,1,0,0,0,0
8186,18,-0.289256,0,1,23.658389,2018-03-11,0,1,0,0,0,0
8187,27,-0.585136,1,0,-0.061575,2018-07-24,0,1,0,0,0,0
8188,29,-0.067346,0,0,-0.061575,2017-10-21,0,1,0,0,0,0


In [ ]:
df_full = pd.read_csv("../data/ml-datasets/full_label.csv")
df_full

,Unnamed: 0,App,Category,Rating,Reviews,Size,Label,Paid,Price,Content Rating,Genres,Last Updated
0,8532,+Download 4 Instagram Twitter,SOCIAL,4.5,40467,22000,1,0,0.00,Everyone,Social,2018-08-02
1,324,- Free Comics - Comic Apps,COMICS,3.5,115,91000,0,0,0.00,Mature 17+,Comics,2018-07-13
2,4541,.R,TOOLS,4.5,259,203,0,0,0.00,Everyone,Tools,2014-09-16
3,4636,/u/app,COMMUNICATION,4.7,573,53000,0,0,0.00,Mature 17+,Communication,2018-07-03
4,5940,058.ba,NEWS AND MAGAZINES,4.4,27,14000,0,0,0.00,Everyone,News & Magazines,2018-07-06
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,6420,감성학원 BL 첫사랑,COMICS,4.4,190,34000,0,0,0.00,Everyone,Comics,2017-11-17
8186,4362,💎 I'm rich,LIFESTYLE,3.8,718,26000,0,1,399.99,Everyone,Lifestyle,2018-03-11
8187,2575,"💘 WhatsLov: Smileys of love, stickers and GIF",SOCIAL,4.6,22098,18000,1,0,0.00,Everyone,Social,2018-07-24
8188,7559,📏 Smart Ruler ↔️ cm/inch measuring for homework!,TOOLS,4.0,19,32000,0,0,0.00,Everyone,Tools,2017-10-21


In [ ]:
df_full['Last Updated'] = pd.to_datetime(df_full['Last Updated'])
df_full = df_full.sort_values(by='Last Updated')
df_full = df_full.drop(columns="Unnamed: 0")

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_full, test_size=0.2, stratify=df_full['Category'], random_state=42)
train_indices = train_df.index
test_indices = test_df.index
df = df.sort_values(by='Last Updated').drop(columns="Last Updated")
train_df = df.loc[train_indices]
test_df = df.loc[test_indices]
X_train = train_df.drop(columns=['Label'])
y_train = train_df["Label"]

X_test = test_df.drop(columns=['Label'])
y_test = test_df["Label"]

In [ ]:
X_train

,Category,Size,Paid,Price,Adults only 18+,Everyone,Everyone 10+,Mature 17+,Teen,Unrated
580,20,0.524414,1,0.827353,0,1,0,0,0,0
6520,26,0.228534,0,-0.061575,0,1,0,0,0,0
6471,14,0.228534,0,-0.061575,0,0,0,0,1,0
4015,30,-1.250865,0,-0.061575,0,1,0,0,0,0
4255,8,0.672354,0,-0.061575,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
5007,5,-0.844030,0,-0.061575,0,0,0,0,1,0
86,18,-0.733075,0,-0.061575,0,0,0,0,1,0
4036,11,-0.622121,0,-0.061575,0,1,0,0,0,0
7395,18,1.818889,0,-0.061575,0,1,0,0,0,0


In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
import numpy as np

# Define the classifiers
lr = LogisticRegression(random_state=42)
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
ada = AdaBoostClassifier()
xgb = XGBClassifier(eval_metric='logloss')

# Define parameter grids
para_knn = {'n_neighbors': np.arange(1, 50)}
para_dt = {'criterion': ['gini', 'entropy'], 'max_depth': np.arange(1, 50),
           'min_samples_leaf': [1, 2, 4, 5, 10, 20, 30, 40, 50, 80, 100]}
params_rf = {'n_estimators': [100, 200, 350, 500],
             'min_samples_leaf': [2, 10, 30, 50, 80, 100]}
params_ada = {'n_estimators': [50, 100, 250, 400, 500],
              'learning_rate': [0.1, 0.001, 0.2, 0.5, 0.8, 1],
              "algorithm":["SAMME"]}
params_xgb = {'n_estimators': [50, 100, 250, 600, 800, 1000],
              'learning_rate': [0.1, 0.001, 0.2, 0.5, 0.8, 1]}

# Function to perform CV and log all results
def cross_validation(model, param_grid, X_train, y_train, search_type='grid', model_name=""):
        if search_type == 'grid':
            search = GridSearchCV(model, param_grid=param_grid, cv=5, return_train_score=True)
        else:
            search = RandomizedSearchCV(model, param_distributions=param_grid, cv=5, return_train_score=True)
        search.fit(X_train, y_train)
        print(f"Best parameters for {model_name}: {search.best_params_}")
        print(f"Best cross-validated score for {model_name}: {search.best_score_}")

cross_validation(knn, para_knn, X_train, y_train, model_name="KNN")
cross_validation(dt, para_dt, X_train, y_train, model_name="Decision Tree")
cross_validation(rf, params_rf, X_train, y_train, model_name="Random Forest")
cross_validation(ada, params_ada, X_train, y_train, model_name="AdaBoost")
cross_validation(xgb, params_xgb, X_train, y_train, search_type='random', model_name="XGBoost")


Best parameters for KNN: {'n_neighbors': 15}
Best cross-validated score for KNN: 0.6785708712538066
Best parameters for Decision Tree: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 80}
Best cross-validated score for Decision Tree: 0.6839101903447633
Best parameters for Random Forest: {'min_samples_leaf': 10, 'n_estimators': 500}
Best cross-validated score for Random Forest: 0.6948988302152659
Best parameters for AdaBoost: {'algorithm': 'SAMME', 'learning_rate': 1, 'n_estimators': 400}
Best cross-validated score for AdaBoost: 0.6785706383449497
Best parameters for XGBoost: {'n_estimators': 250, 'learning_rate': 0.1}
Best cross-validated score for XGBoost: 0.6857434159577502
